In [53]:
intention = '''Create an easy/safe API to parse/hold/yield constructions of pydantic docx class objects.
In theory this can be generalized and abstracted since the classes are largely absracted. 
However since this is API based, I will just focus on solutions for THIS dictionary task. These can be expanded later, or just have other APIs added
Final objective is have a tool to pass complex commands/instructions, and have it yield a pandas dataframe of the results.
Intermediate steps may be yield a list of lists
I am not yet sure if there is benefit trying to have the whole dictionary in one notional object. 
I don't think so, since each operation is psuedo atomic, and any aggregate actions are intended to be done in something like pandas.
Eventually I would want a loop for pandas operations to easily feed updates to the data. 
'''
conclusion = '''create a pular dictionary specifc datastructure to handle the aggregate paragraphs. 
This will only be per lemma, 
as any aggregations from roots and relations of that sort can be managed in pandas and created by simple inheretance/reference
'''

In [54]:
from typing import Optional, Dict, List, Any, Union, Tuple
import pydantic
from pydantic import ValidationError, validator, root_validator, Field, constr
from pydantic_docx import Docx_Paragraph_and_Runs, read_docx #type:ignore
from pydantic_docx_processor import create_sized_dataframe, expand_dataframe #type:ignore
import re
import json
from itertools import compress, chain
from datetime import datetime
import pandas as pd
import numpy as np
from functools import partial

In [55]:
import pickle
with open('feature_Frames_and_Indexes.pkl', 'rb') as file:
    # Call load method to deserialze
    output = pickle.load(file, encoding='utf-8')
(
parsed_object_list, #:List[Tuple[int,Docx_Paragraph_and_Runs]]
parsed_object_lookup, #:Dict[int,Docx_Paragraph_and_Runs] = dict(parsed_object_list)
doc_para_count, #: int = int(parsed_to_dict['total_encountered_paragraphs']) #type: ignore
char_counts, #: Counter = parsed_to_dict['char_counts'] #type: ignore
rootFrame,#:pd.DataFrame
rootsubpieceFrame, #:pd.DataFrame
lemmaFrame, #:pd.DataFrame
nonentityParaFrame, #pd.DataFrame
cleanerOutcomesDf #pd.DataFrame
    ) = output


In [56]:
#first check the frame containing the success/failure boolean for the cleaner operation that was performed on the data in pydantic_docx ingestion
print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
cleanerOutcomesDf.dropna(subset=['paragraphIndex'],inplace=True)
print(cleanerOutcomesDf['cleaner_success_outcomes'].unique())
#results: all values with a valid paragraph will have a value in 'paragraphIndex'. ie paragraphs that did not have a suppressed error during class assignment. By default these are of only whitespace.
#when parsing, the default was used, so any all whitespace paragraph will be nan in paragraphIndex.
#when these values are dropped, the boolean column (indicating the successful post-processing of already created pydantic_docx class objects) yields only True values.
#that was a complex way of saying "none of the paragraphs had an error occur after their creation"...
# ...and therefor, all are "good" for the next steps

[nan True]
[True]


The following merge routine merges together the parents of one level of the heirarchy with their children. 
The parents then have their values forward-filled / duplicated to account for their headship over the child entities
Each frame has an identity column with a boolean value indicating that index is the location of that entity.
These are not needed for propogating headship and or child relationships, and these will not be forward-filled.
As such, these will be removed from the entity frames before they are added to the entity heirarchy frame

Precedence is as follows:
root > subroot > lemmas > paragraphs

However, the child relationship is non-exclusive to direct 'parents', such that a parent 'entity' may have a dhild from any of the levels below them.
That said, the dictionary description and general structure would indicate paragraphs should only be direct children of Lemmas
>paragraphs being defined as any line without an 'entity' such as root, subroot, or lemma.
>any that are not would be a deviation. 
Therefor, any such instance needs to be captured and assessed, and should not be coerced into some other neighbors "family" by default, if at all.

As a result, our routine for creating hierarchys must allow for any possible/permitted parent be available to match any children.

So...

Subroots:      possible parents are:      Roots
Lemmas:        possible parents are:      Roots, Subroots
Paragraphs:    possible parents are:      Roots, Subroots, Lemmas

*Note: the term 'subroot' is sometimes described as 'root-subpiece', and these names may be understood interchangibly. The reason for the two names stems from two competing 'theories' or impressions of the nature of these entities. Initially I used the term subroot as a description of their subordinate position. The term subpiece has begun entering my vocabulary after discovering that many (if not all) of these are 'infixes', a core component of the Pular language. As a result any of these that are infixes are more than just a manual 'byte pair encoding' or common break point, but have specific meaning of their own.*

In [57]:
all_entity_identity_frame = rootFrame[['is']].merge(rootsubpieceFrame[['is']],how='outer', on='index').merge(lemmaFrame[['is']],how='outer',on='index')
all_entity_identity_frame.columns = ['is_root', 'is_root_subpiece','is_lemma']

# rootFrame contains the permitted parents for subroots.
# print(rootFrame)
rootFrame['is']='root'
root_parent = rootFrame.copy()
#next the frame of permitted parents for lemmas is created by combining/overlaying roots and subroots
rootsubpieceFrame['is'] = 'root_subpiece'
any_roots = root_parent.combine_first(rootsubpieceFrame)

#finally create the permitted parents of paragraphs (though we will want to examine any paragraphs subordinate directly to a root. These may indicate some datacleaning is needed, or may need to be removed entirely)
lemmaFrame['is'] = 'lemma'
anyEntityFrame = any_roots.combine_first(lemmaFrame)

anyEntityFrame.sample(10)

,paragraphIndex,is,text,mask,run_text_list
index,,,,,
29717,29717.0,lemma,woɗeede (2),"[True, False, False]","[woɗeede (2) , v.pv , F [wojjude]]"
19872,19872.0,root,NDUPPAARA,[True],[NDUPPAARA]
25069,25069.0,lemma,sippugol,"[True, False, False]","[sippugol , (ngol) , Y]"
9553,9553.0,lemma,haasidaaku,"[True, False, False]","[haasidaaku , (ngu,o) , DZu Z<->]"
4063,4063.0,lemma,denkitaade,"[True, False, False]","[denkitaade , v.mv+ext , Z <FT,M>]"
18096,18096.0,lemma,mulgiɗidde,"[True, False, False]","[mulgiɗidde , v.av+ext , Z <DO,M>]"
16034,16034.0,root,LEFOL,[True],[LEFOL]
5189,5189.0,root,ƊIRNOL,[True],[ƊIRNOL]
32249,32249.0,lemma,"ƴeewude, ƴeewde","[True, False, False]","[ƴeewude, ƴeewde , v.av , DFCZ C<->,Z<FJ,F..."


Now these frames are set up, we can begin the routine. 

At each step, 
   >we merge the possible parents frame with the specific child frame.
   >then we forward-fill the parent portion of the frame to indicate the headship for every record.
   >- we want to 'unnest' this datastructure, and so we need to move from 'dictionary' style to 'record' style.
   >- records require all pertinent infromation be present inside the record (records are usually defined as a single row in most industries)
   >- so forward filling all possible parents will allow 'parentage' be infered by position / precedence on the page.
   >  - a lemma is the child of the nearest 'parent' root UP the page from it, regardless if a root is directly below it.
   >     - therefor forward-fill will allow these inferred relations be directly documented in each record, each row. 
   >     - SQL design says don't repeat yourself, but here we WANT to duplicate the parent value onto each child.
   >     - This allows us to safely remove a 'child' from its context on the page without loosing who it's parents are.
   >- then we merge in the 'frame' of next parent group, then child group, and repeat the forward filling 
   > 
   > *the process is iterative/recursive.*

However, the parent/child relationships happen to align such that each next lower group of parents is composed of the parents + the child from the previous iteration...
As a result we actually have a much simpler structure: we only need to merge each parent grouping directly, and forward fill all of those

We will have some extra duplication from top parents being copied down on each level between them and their children, but we will solve this after we create it.

The extra reason to group it this way is it does allow for a root to be declared on the same paragraph/line as a lemma, which is fairly common from manual review of the dictionary word document

In [59]:
#subroot-parent columns -> merged with <- lemma-parents columns
entityInheritanceFrame = root_parent.merge(any_roots, on='index',how ='outer',suffixes=['_main_root',''],sort=True, validate='one_to_one')

#'subroot-parent' and 'lemma-parent' columns -> merged with <- 'paragraph-parents' columns
entityInheritanceFrame = entityInheritanceFrame.merge(anyEntityFrame, on='index',how ='outer',suffixes=['_any_root','_any_entity'],sort=True, validate='one_to_one')

#before forward-fill, but we want to remove "non-paragraph" rows from our data (if present), since we don't care about empty paragraphs as children to our 'parents' (remembering that empty paragraphs are all np.nan)
#this should have no np.nan values in the final paragraph-parents column, but lets check before we accidentally hide the evidence
# print(entityInheritanceFrame.shape) >>> (15266, 12)
entityInheritanceFrame = entityInheritanceFrame.dropna(subset=['paragraphIndex_any_entity'])
# print(entityInheritanceFrame.shape) >>> (15266, 12)

#forward-fill np.nan values in the whole dataframe
entityInheritanceFrame = entityInheritanceFrame.fillna(method = 'ffill') #this can be skipped here, as it is redundant, but its a valuable midway point, and informative to review and check our assumptions

#'subroot-parent' and 'lemma-parent' and 'paragraph-parents' columns -> merged with <- 'paragraphs' columns
#note the 'paragraphs' paraFrame contains valid and invalid paragraphs, like the cleaner-output frame we dealt with above.
#it specifically only contains the indexes of paragraphs that are valid AND did not contain on of the entities we've been dealing with (roots, subroots, lemmas)
nan_free_paraFrame = nonentityParaFrame.dropna() #this will remove all non-paragraph records
allParagraphsInheritanceFrame = entityInheritanceFrame.merge(nan_free_paraFrame, on='index',how ='outer',suffixes=['','_paragraphs'],sort=True, validate='one_to_one')                                       
#since we dropped np.nan values above, the only nan values should be introduced by the paragraph merge. Where present in the paragraph columns, these will be the declaration columns of the lemmas and roots.
#to solve that we will forwardfill all columns BUT the new paragraphIndex column, which will fill in the parents for the new paragraphs.
allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns] = allParagraphsInheritanceFrame.loc[:,entityInheritanceFrame.columns].fillna(method = 'ffill')

#here, we can choose to dropna in the dataframe, which will remove all records that do not have a child paragraph in them.
#this will be necessary for creating records to match to the wordnet, since the paragraphs will be our glosses we need.
#however, any record that has a lemma, but does not have paragraphs still has valuable information, so we don't want to discard these completely, we just can't use them directly to search the wordnet.

# identify_condition = partial(lambda x: bool(re.match('is_',x)))
# subroot_condition = partial(lambda x: bool(re.search('_root_subpiece',x)) or bool(re.search('_anyroot',x)))

# entityInheritanceFrame
allParagraphsInheritanceFrame
#TODO break oneliners out, break roots out of those, check numbers with other checks

,paragraphIndex_main_root,is_main_root,text_main_root,mask_main_root,run_text_list_main_root,paragraphIndex_any_root,is_any_root,text_any_root,mask_any_root,run_text_list_any_root,paragraphIndex_any_entity,is_any_entity,text_any_entity,mask_any_entity,run_text_list_any_entity,paragraphIndex
index,,,,,,,,,,,,,,,,
4,4.0,root,A,[True],[A],4.0,root,A,[True],[A],4.0,root,A,[True],[A],NaN
5,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",NaN
6,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",6.0
7,4.0,root,A,[True],[A],4.0,root,A,[True],[A],5.0,lemma,a,"[True, False, False]","[a , prn,sbj,sf , DFZH Z<->]",7.0
8,4.0,root,A,[True],[A],4.0,root,A,[True],[A],8.0,lemma,-a,"[True, False, False]","[-a , suf,pos , F]",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32502,32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32499.0,root,ƳUUƳ-,[True],[ƳUUƳ-],32500.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",32502.0
32503,32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],NaN
32504,32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503.0,root,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32504.0,lemma,ƴuuƴude,"[True, False, False]","[ƴuuƴude , v.av , D]",NaN


In [15]:
# print(rootFrame.head())
print(rootFrame.columns)
entityInheritanceFrame = expand_dataframe(rootFrame,doc_para_count)
print(rootFrame.shape)
# entityInheritanceFrame = entityInheritanceFrame.join(rootFrame,on = 'index', how = 'outer')
entityInheritanceFrame

Index(['paragraphIndex', 'cleaner_success_outcomes', 'is_root', 'root_text',
       'root_mask', 'root_run_text_list'],
      dtype='object')
(6381, 6)


,paragraphIndex,cleaner_success_outcomes,is_root,root_text,root_mask,root_run_text_list,index
0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,NaN,NaN,NaN,NaN,NaN,NaN,1
2,NaN,NaN,NaN,NaN,NaN,NaN,2
3,NaN,NaN,NaN,NaN,NaN,NaN,3
4,4.0,True,True,A,[True],[A],4
...,...,...,...,...,...,...,...
32502,NaN,NaN,NaN,NaN,NaN,NaN,32502
32503,32503.0,True,True,ƳUUƳ- (2?),[True],[ƳUUƳ- (2?)],32503
32504,NaN,NaN,NaN,NaN,NaN,NaN,32504
32505,NaN,NaN,NaN,NaN,NaN,NaN,32505


In [3]:
outputs = [
   
   'txt_FulaLemmas',
   'txt_FulaRoots',
   
   #extra
   'txt_FulaAnnotations',
   #gloss derivatives
   'txt_FulaSenseEnglish',
   'txt_FulaSenseFrench',
   'txt_FulaSenseClassifications',
   'txt_FulaSenseEnglishAnnotations',
   'txt_FulaSenseFrenchAnnotations',

   #lemma derivative
   'txt_FulaDialects',
   'txt_FulaPOSTags',
   'txt_FulaSynonyms',


   #agg
   'txt_FulaInParenthesis',
   'txt_POS',
   'txt_RootOrigins'
]

In [4]:
# noun_patterns = [r'n\.',r"(?<=\()[^\)]+",r"\/"]
# nounPatternRegex = re.compile('|'.join([p for p in noun_patterns]))
# print(nounPatternRegex)
# s = 'n. dfhjkgqh (wsh) askldjhf / asdfhjkkh'
# print(re.findall(nounPatternRegex,s))
# print(nounPatternRegex.findall(s))

# if /, the next word is a plural. The plural also then usually has a (el) class after it. commas are multiple version
# so each lemma noun may have a (class). The lemma may have one or multiple plurals, which will have a modified (class)
# so have a nouns list? nouns may then be 
# [('n.',{bool}), #only needed if this gets added to all POS? Or maybe theres information in its use?
# ([[{word}],{Optional[class]}]),  #singular
   #([[{word}],{Optional[class]}])]   #plural

# so nounStruct[0][1] is the flag for noun?
   # one POS table, and noun will get added there. But the presence of a noun indicates to look here
# classes may be found [sClass[1] for sClass in nounStruct[1]] will give all classes of singular, [sClass[1] for sClass in nounStruct[1]]

re.compile('n\\.|(?<=\\()[^\\)]+|\\/')
['n.', 'wsh', '/']
['n.', 'wsh', '/']


In [5]:
# now = datetime.now()
# current_time = now.strftime("%Y-%m-%d_-_%H-%M-%S")
# print(f"Experiment time: {current_time}\nExperiment note: {experiment}\n\n")

In [30]:
def getAssert(dct,val,typ:type, strict = True) -> Any:
   out = dct.get(val,None)
   if strict and typ is type(None):
      raise NotImplementedError(f'this get/assert function cannot "get" None values safely, and a {val} type was passed for dict {dict}')
   if strict and out is None:
      raise KeyError(f"the provided dict {dct} did not have the key: {val}")
   assert isinstance(out,typ), f"the provided dict {dct} did not give {val} with the expected type: {typ}, but instead {out}"
   return out
   
def trustyGet(obj:Docx_Paragraph_and_Runs, feat: str, silent_return = True) -> str:
      if not silent_return:
         output : str = getattr(obj,feat,'')
         if len(output) > 0:
            return output
         else:
            raise AttributeError(f'trusty getter could not find: {feat}')
      else:
         output : str = getattr(obj,feat,'')
         return output

In [36]:
class dict_entry(pydantic.BaseModel): 
   '''
   #True is used as default value, and must be changed to False if the feature is not found. This will catch not implemented, and incomplete updates
   '''
   irregularities: List[str] = []
   lemma_index: int 
   paragraphs: List[Docx_Paragraph_and_Runs] = Field(...,min_items = 1)
   lemma: str = Field(...,min_length = 1) #article
   root: str
   root_subpiece: Union[str,bool] = True #these will not be updated after initialization
   root_metadata: Dict[str,Any]  #index,root_text, root_meta_data #article 
   root_origin: Union[str,bool] = True #these will not be updated after initialization
   #Paragraph Derivatives: Glosses and Annotations
   lemmaLine_runs: List[str] #these will not be updated after initialization
   englishGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   frenchGlossLine_runs: Union[List[str],bool] = True #these will not be updated after initialization
   FulaAnnotations_runs: Union[List[List[str]],bool]= True #these will not be updated after initialization

   #Gloss Derivatives: English Senses
   FulaSenseEnglish: Union[List[str],bool] = True #list of "senses" split by semicolons #article
   FulaSenseEnglish_Count: Union[int,bool] = True #number of senses in english #aka FulaSenseClassifications #article
   FulaSenseEnglish_Annotations: Union[List[str],bool] = True #contains the annotations (in parenthesis) for a given sense, if any #article 
   FulaSenseEnglish_Synonyms: Union[List[str],bool] = True #holds bracket text for a sense, suspected all synonyms. These may all occur at the end, and may be redundant with the synonyms provided at the head of the entry
   FulaSenseEnglish_unusedContent: Union[List[str],bool] = True #holds any run text that is not contained in the above features
   
   #Gloss Derivatives: French Senses
   FulaSenseFrench: Union[List[str],bool] = True #article
   FulaSenseFrench_Count: Union[int,bool] = True #aka FulaSenseClassifications
   FulaSenseFrench_Annotations: Union[List[str],bool] = True
   FulaSenseFrench_Synonyms: Union[List[str],bool] = True
   FulaSenseFrench_unusedContent: Union[List[str],bool] = True
   
   #lemma derivative
   lemmaLine_unusedContent: List[Tuple[int,str]] #= Field(...) #int is index for run to allow tracing as entries are removed
   FulaDialects: Union[List[str],bool] = True #article
   FulaPOSTags: Union[List[str],bool] = True #article
   FulaPOSClass: Union[List[str],bool] = True #Noun, Adj, Verb, Adv, Prn, ..., Complicated, Indeterminate
   FulaNoun_NounsAndClass: Union[List[Tuple[str,str]],bool] = True #Pular has unique noun classes. Lemma will be copied here beside its class ("noun", "nounclass"), and any additional singular forms will be in additional tuples in this same list
   FulaNoun_PluralsAndClass: Union[List[Tuple[str,str]],bool] = True #Dict provides plurals for nouns. Tuples will have ("noun", "nounclass")
   FulaSynonyms: Union[List[str],bool] = True #article
   FulaCrossRef: Union[List[str],bool] = True
         # FulaVerbClass:
   
   class Config:
      validate_all = True
      validate_assignment = True
      smart_union = True  
      extra = 'forbid'

   @root_validator(pre=True)
   def _validate_and_build(cls, values: Dict[str,Any]) -> Dict[str, Any]:
      # print(values)
      newValues: Dict[str,Any] = {}
      newValues['irregularities'] = []

      ###PARAGRAPHS CHECK###
      print(values)
      newValues['paragraphs'] = getAssert(values,'paragraphs',list) #these come in as tuples(ind,para)
      #TODO have more rigorous checks on para since so much depends on that even in this validation
      assert isinstance(newValues['paragraphs'][0],tuple), 'currently paras are packaged in a tuple with their original doc index'
      newValues['paragraphs'] = [p for i,p in newValues['paragraphs']]
      if len(newValues['paragraphs']) == 0:
         raise ValueError('given invalid paragraphs of zero length')
      else: #logic for subsidary paragraphs
         lemma_line = trustyGet(newValues['paragraphs'][0],feat = 'run_text', silent_return=False)
         newValues['lemmaLine_runs'] = lemma_line
         if len(newValues['paragraphs']) == 1:
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 2:
            newValues['irregularities'].append('What:ambiguous, Where:paragraphs, Why: only one')
            newValues['englishGlossLine_runs'] = False
            newValues['frenchGlossLine_runs'] = False
         elif len(newValues['paragraphs']) == 3:
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
         else: 
            newValues['englishGlossLine_runs'] = newValues['paragraphs'][1].get_run_text()
            newValues['frenchGlossLine_runs'] = newValues['paragraphs'][2].get_run_text()
            newValues['FulaAnnotations_runs'] = newValues['paragraphs'][3:].get_run_text()
      
      
      try: ###LEMMA CHECK###
         #reading in lemma results from first pass of pydantic parser
         lemma_index, lemma_mask, lemmaLine_runs = getAssert(values,'lemma',tuple)
         #checking for correct structure
         lemma_matched_runs = list(compress(lemmaLine_runs,lemma_mask))
         if len(lemma_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Lemmas, Why: the merge routines should aggregate adjacent runs with same features. Multiple Lemma runs should not be possible if Bold is contiguous')
         lemma_text = ''.join(chain(lemma_matched_runs)).strip()
            #TODO have better control for expected structure that these runs should be adjacent (and only should be one)
         #saving values. These will have to pass type check of declared attribute types for validation to succeed
         newValues['lemma'] = lemma_text
         newValues['lemma_index'] = lemma_index
         newValues['lemmaLine_runs'] = lemmaLine_runs
         used_run_mask = lemma_mask
      except Exception as e:
         #exception lemma check
         raise RuntimeError('failed lemma check') from e

      try: ###ROOT CHECK###
         #reading in root results from first pass of pydantic parser
         #checking for correct structure
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root'] = root_text
         newValues['root_metadata'] = {'root_index': root_index, 'root_runs':rootLine_runs}
      except Exception as e:
         #exception ROOT check
         raise RuntimeError('failed ROOT check') from e

      try: ###ROOT-Subpiece CHECK###
         #iterating in case a subroot is present
         root_index, root_mask, rootLine_runs = getAssert(values,'root',tuple)
         root_matched_runs = list(compress(rootLine_runs,root_mask))
         if len(root_matched_runs) > 1:
            newValues['irregularities'].append('What:Unexpected, Where: Root-Subpiece, Why: the merge routines should aggregate adjacent runs with same features. Multiple Root runs should not be possible if Fontsize is contiguous and unique')
         root_subpiece_text = ''.join(chain(root_matched_runs)).strip()
         if root_index == newValues['lemma_index']:
            newValues['irregularities'].append('What:inconsistent, Where: Lemmas and Root-Subpiece, Why:normally root and lemma are on different lines. This has them sharing, which may indicate a lack of other content')
            used_run_mask = [any(run) for run in list(zip(used_run_mask,root_mask))]
            rootLine_runs = False
         newValues['root_subpiece'] = root_subpiece_text
         newValues['root_metadata'] = {'root_subpiece_index': root_index, 'root_subpiece_runs':rootLine_runs}
      except AssertionError as e:
         newValues['root_subpiece'] = False
      except Exception as e:
         #exception ROOT-Subpiece check
         raise RuntimeError('failed ROOT-Subpiece check') from e

      #determining lemmaLine_unusedContent
      unused_run_mask = [not r for r in used_run_mask]
      print(list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask)))
      newValues['lemmaLine_unusedContent'] = list(compress(enumerate(newValues['lemmaLine_runs']),unused_run_mask))
      
      return newValues

   def parse_senses(self):
      #TODO
      return

   def parse_glossRemainder(self):
      #TODO
      return

   def parse_lemmaLine(self):
      #TODO
      return

   def parse_lemmaLineRemainder(self):
      #TODO
      return

   def get_rootOrigin(self):
      #TODO
      return
 
   def give_entryText(self, joiner = '\t') -> str: #article
      return joiner.join([para.trustyGet('para_text') for para in self.paragraphs])

   
      
            
# dict_entry.parse_obj({'paragraphs':[b for a,b in parsed_object_list[5:8]]})

# incoming = {
#    'paragraphs': List[Docx_Paragraph_and_Runs],
#    'root': List[int,str,List[bool]],
#    'lemma': List[int,str,List[bool]],
#    # 'pos': 
# }

# [
#    ['paras'], #obj
#    ['parsed_paras'], #pydocx
#    ['roots'], #name, fill forwards
#    ['lemmas']  #text, fill forwards
# ]

In [8]:
# import pickle
# with open('parsed_objectClass_outcomes_dict.pkl', 'rb') as file:
#     # Call load method to deserialze
#     parsed_objectClass_outcomes_dict = pickle.load(file, encoding='utf-8')

# parsed_object_list = parsed_objectClass_outcomes_dict['parsed_object_list'] 
# para_text_lookup = parsed_objectClass_outcomes_dict['para_text_lookup'] 
# root_ind_list = parsed_objectClass_outcomes_dict['root_ind_list'] 
# subroot_ind_list = parsed_objectClass_outcomes_dict['subroot_ind_list'] 
# lemma_ind_list = parsed_objectClass_outcomes_dict['lemma_ind_list'] 
# root_and_lemma_one_line = parsed_objectClass_outcomes_dict['root_and_lemma_one_line'] 
# root_lookup = parsed_objectClass_outcomes_dict['root_lookup'] 
# lemma_lookup = parsed_objectClass_outcomes_dict['lemma_lookup'] 
# # char_counts = parsed_objectClass_outcomes_dict['char_counts']

In [9]:
# docx_filename = "pasted_docx page 1.docx"
docx_filename = "Fula_Dictionary-repaired.docx"
parsed_to_dict = read_docx(docx_filename)

outcomes_dict = monolith_root_and_lemma_processor(parsed_to_dict['parsed_object_list'],parsed_to_dict['char_counts'])

parsed_object_list = outcomes_dict['parsed_object_list'] 
para_text_lookup = outcomes_dict['para_text_lookup'] 
root_ind_list = outcomes_dict['root_ind_list'] 
subroot_ind_list = outcomes_dict['subroot_ind_list'] 
lemma_ind_list = outcomes_dict['lemma_ind_list'] 
root_and_lemma_one_line = outcomes_dict['root_and_lemma_one_line'] 
root_lookup = outcomes_dict['root_lookup'] 
lemma_lookup = outcomes_dict['lemma_lookup'] 
char_counts = outcomes_dict['char_counts']

In [ ]:
# #Creating lists of indexes for use in remaining processing

# # # print(len(parsed_object_list)) #32040
# #note this parsed paras are packaged with the original para number in the docx
# #therefore 32507 is the index to use, but len of that IS 32040, since ~500 were empty paragraphs that were not parsed
# parsed_para_indexes = [i for i,p in parsed_object_list]

# lim = 32507
# # rootInds = [x for x in root_ind_list if x < lim]
# rootInds=root_ind_list
# # subRootInds = [x for x in subroot_ind_list if x < lim]
# subRootInds=subroot_ind_list
# # lemmaInds = [x for x in lemma_ind_list if x < lim]
# lemmaInds=lemma_ind_list

# allroots = rootInds.copy()
# allroots.extend(subRootInds)
# allroots = sorted(allroots)
# allEntities = allroots.copy()
# allEntities.extend(lemmaInds)
# allEntities = sorted(set(allEntities))
# print('allEntities: ',len(allEntities))


# # all_paras = [i for i,obj in parsed_object_list if i < lim]
# normal_para = parsed_para_indexes.copy()
# [normal_para.remove(i) for i in allEntities]
# print('normal_para: ',len(normal_para))

# # normal_para = [x for x in parsed_para_indexes if all([x not in rootInds, x not in lemmaInds, x not in subRootInds])]
# # print(normal_para)
# # print(allroots)
# root_aligned_lemmas = list(closest(pairwise(allroots), lemmaInds))
# lemma_aligned_paras = list(closest(pairwise(lemmaInds),normal_para))

In [39]:
# import pickle
# with open('entity_index_schema.pkl', 'rb') as file:
#     # Call load method to deserialze
#     entity_index_schema = pickle.load(file, encoding='utf-8')

In [ ]:
# for r,v1 in entity_index_schema.items():
#    root = (r,root_lookup[r][1],root_lookup[r][2])
#    if isinstance(v1,dict):
#       for k,v in v1.items():
#          if k in subroot_ind_list:
#             subroot

In [38]:
# result = []
# for i in parsed_object_list[5:15]:
#    result.append(dict_entry({i))
# print(root_lookup[4])
# print(para_text_lookup[3])
# root_container = [(ind, mask, rootLine_runs) for ind, mask, rootLine_runs in getAssert(values,'root',list)]
shorter_obj_list = parsed_object_list[:300]
# for r, l
inds = [8,9,10]

dct = {
   'paragraphs':[o for o in shorter_obj_list if o[0] in inds],
   'root': (4,root_lookup[4][1],root_lookup[4][2]),
   'lemma': (inds[0],lemma_lookup[inds[0]][1],lemma_lookup[inds[0]][2])
}
dict_entry.parse_obj(dct)

# {'paragraphs': [(9, 
#    Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)')), (10, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)'], run_bold=[None], para_text='ton, ta, tes (seulement pour certains substantifs, tels ceux qui se rapportent aux membres de la proche famille)')), 
#    (11, Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=182880, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=11, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['aan  ', 'prn,ind  ', 'DFZH  [an]:Z<->'], run_bold=[True, None, None], para_text='aan  prn,ind  DFZH  [an]:Z<->'))], 'root': [(4, [True], ['A'])], 'lemma': (5, [True, False, False], ['a  ', 'prn,sbj,sf  ', 'DFZH  Z<->'])
#    } did not have the key: paragraphs"

{'paragraphs': [(8, Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=182880, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=8, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['-a  ', 'suf,pos  ', 'F'], run_bold=[True, None, None], para_text='-a  suf,pos  F')), (9, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)')), (10, Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=274320, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seulement pour certains substantifs, te

dict_entry(irregularities=[], lemma_index=8, paragraphs=[Docx_Paragraph_and_Runs(run_italic=[None, True, None], para_left_indent=116128800, run_font_name=['TmsRmn 10pt', 'TmsRmn 10pt', 'Helv 8pt'], paragraph_enumeration=8, run_font_size_pt=[None, None, 8.0], para_first_line_indent=-91440, run_text=['-a  ', 'suf,pos  ', 'F'], run_bold=[True, None, None], para_text='-a  suf,pos  F'), Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=174193200, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=9, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['your (sg.) (only with certain nouns such as those which refer to close family members)'], run_bold=[None], para_text='your (sg.) (only with certain nouns such as those which refer to close family members)'), Docx_Paragraph_and_Runs(run_italic=[None], para_left_indent=174193200, run_font_name=['TmsRmn 10pt'], paragraph_enumeration=10, run_font_size_pt=[None], para_first_line_indent=-91440, run_text=['ton, ta, tes (seul